In [ ]:
import Pkg; 
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [12]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [13]:
# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

function rk4(model, ode, x, u, dt)
    k1 = dt*ode(model, x, u)
    k2 = dt*ode(model, x + k1/2, u)
    k3 = dt*ode(model, x + k2/2, u)
    k4 = dt*ode(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function get_jacobians(model, dynamics, X̄, Ū , dt)
    N_mpc = length(X̄)
    A = [FD.jacobian(_x -> rk4(model, dynamics, _x, Ū[i], dt), X̄[i]) for i = 1:(N_mpc - 1)]
    B = [FD.jacobian(_u -> rk4(model, dynamics, X̄[i], _u, dt), Ū[i]) for i = 1:(N_mpc - 1)]
    return A, B
end

get_jacobians (generic function with 1 method)

In [216]:
# ====================
# Explicit model params (working)
# ====================
@variables x[1:6] u[1:2] 
x = collect(x); u = collect(u)
x0 = [1; 2; 3; 4; 5; 6]; u0 = [7; 8]
# Planar Quadrotor Dynamics
function dynamics(x, u)
    model = (g = 9.81, #m/s^2
        m = 1.0, #kg 
        ℓ = 0.3, #meters
        J = 0.2*1*0.3*0.3) 
    θ = x[3]
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4]; x[5]; x[6]; ẍ; ÿ; θ̈]
end
function rk4(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function midpoint(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end

sym_jac_A = Symbolics.jacobian(rk4(x, u), x)
sym_jac_B = Symbolics.jacobian(rk4(x, u), u)
# Sub value to validate with ForwardDiff
sub_dict = Dict()
[sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
val_jac_A = substitute.(sym_jac_A, (sub_dict,))
val_jac_B = substitute.(sym_jac_B, (sub_dict,))
fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0)
norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B)

val_jac_A = substitute.(sym_jac_A, (sub_dict,)) = Num[1 0 -0.07403629041099907 0.09999999999999999 0 -0.002464323899799397; 0 1 0.00487270388811441 0 0.09999999999999999 0.0004200352044805546; 0 0 1 0 0 0.09999999999999999; 0 0 -1.4526144530017426 1 0 -0.0712251548891752; 0 0 0.25260670224641213 0 1 0.020387966336526807; 0 0 0 0 0 1]


1.597872624000063e-17

In [224]:
# ====================
# Generate C code for jacobians
# ====================
fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianA, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianB, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end

In [228]:
# ====================
# Generate C code for dynamics
# ====================
dynamics_rk4 = rk4(x ,u)
dynamics_rk4 = build_function(dynamics_rk4, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Dynamics_RK4, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics_rk4.txt","a") do io
        println(io, dynamics_rk4)
end

In [ ]:
# ====================
# Symbolic model params
# not working 
# ====================
@variables g, m, ℓ, J
model = (g=g, m=m, ℓ=ℓ, J=J)
x = collect(x)
u = collect(u)
@variables x[1:6] u[1:2] 

# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end
function rk4(model, x, u)
    dt = 0.1
    k1 = dt*dynamics(model, x, u)
    k2 = dt*dynamics(model, x + k1/2, u)
    k3 = dt*dynamics(model, x + k2/2, u)
    k4 = dt*dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4) 
end
# sym_jac_A = Symbolics.jacobian(dynamics(model, x, u), x)
# sym_jac_B = Symbolics.jacobian(dynamics(model, x, u), u)
sym_jac_A = Symbolics.jacobian(rk4(model, x, u), u)

In [18]:
let
dt = 0.05  
h = dt
Nx = 6     # number of state
Nu = 2     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
    
# Quadrotor model parameters
model = (g = 9.81, #m/s^2
        m = 1.0, #kg 
        ℓ = 0.3, #meters
        J = 0.2*1*0.3*0.3) 

# Thrust limits
umin = [0.2*model.m*model.g; 0.2*model.m*model.g]
umax = [0.6*model.m*model.g; 0.6*model.m*model.g]

x_hover = zeros(6)
u_hover = [0.5*model.m*model.g; 0.5*model.m*model.g]
    
#Linearized dynamics for hovering
x_hover = zeros(6)
u_hover = [0.5*model.m*model.g; 0.5*model.m*model.g]
A = FD.jacobian(_x -> rk4(model, dynamics, _x, u_hover, dt), x_hover);
B = FD.jacobian(_u -> rk4(model, dynamics, x_hover, _u, dt), u_hover);
    
end

rk4(model, dynamics, x_hover, u_hover, dt) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0